# Machine Offsets from Least Squares

This notebook will call an arbor probing part program, then solve for the best fit machine offsets using the probing data.

You will need the ruby probe mounted in the collet, and an empty arbor clamped in the spindle.

## Probing part program

Here we will call a part program to probe the arbor in the spindle.

    Note: to run the cell use SHIFT + ENTER

In [ ]:
#Run the probing part program synchronously
import os.path
from amcnc import program

status = program.run(os.path.abspath(r'probe_arbor.pp'))

## Solver

Now that we have a set of probing data, we can use this to solve for the best fit machine offsets.

First we will read the output text file and visualise the data with a 3D plot.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import exp, loadtxt, pi, sqrt, sin, cos
import scipy
from scipy.stats import norm

In [ ]:
#read the measured probing data
data = loadtxt('P:/TG7/misc/arbor_probing_data.txt')

x = data[:, 0]
y = data[:, 1]
z = data[:, 2]
a = data[:, 3]
c = data[:, 4]

In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D

Axes3D = Axes3D  # pycharm auto import
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')

ax.scatter3D(data[:,0], data[:,1], data[:,2])

ax.set_xlabel('X (mm)')
ax.set_ylabel('Y (mm)')
ax.set_zlabel('Z (mm)')

Now we will define a function **arbor_leastsq** which models the machine including the parameters of intrest, then we use **scipy.optimize.leastsq** to solve.

In [ ]:
def arbor_leastsq(vars, x, y, z, a, c):
    """calculate the distance from the spindle centerline vector to the contact point. 
    Distance is sum of arbor and probe ball radius"""
    
    ball_runout = vars[0]
    ball_runout_angle = vars[1]
    radius = vars[2]
    spindle_centerline_offset = vars[3]
    x_offset = vars[4]
    y_home_offset = vars[5]
    z_home_offset = vars[6]
    c_home_offset = vars[7]
    
    ball_runout_angle_radians = (ball_runout_angle/360.0)*2*pi
    c_home_offset_radians = (c_home_offset/360.0)*2*pi
    a_angle_radians = (a/360.0)*2*pi

    ball_x = x + x_offset
    ball_y = y + y_home_offset + ball_runout*sin(a_angle_radians + ball_runout_angle_radians)
    ball_z = z + z_home_offset + ball_runout*cos(a_angle_radians + ball_runout_angle_radians)

    c_angle_radians = (c/360.0)*2*pi

    #find equation of the c-axis vector
    ui = -sin(c_angle_radians + c_home_offset_radians)
    uj = cos(c_angle_radians + c_home_offset_radians)
    uk = 0

    #coords of the spindle axis origin
    x0 = cos(c_angle_radians + c_home_offset_radians)*(-spindle_centerline_offset)
    y0 = sin(c_angle_radians + c_home_offset_radians)*(-spindle_centerline_offset)
    z0 = 0

    #find the closest point on the line
    point_x = x0 + ((ball_x-x0)*ui+(ball_y-y0)*uj+(ball_z-z0)*uk)*ui;
    point_y = y0 + ((ball_x-x0)*ui+(ball_y-y0)*uj+(ball_z-z0)*uk)*uj;
    point_z = z0 + ((ball_x-x0)*ui+(ball_y-y0)*uj+(ball_z-z0)*uk)*uk;
    
    #the error for each point is the distance from the closest point on the c-axis vector, minus the current value for radius
    #i.e. the solver includes the radius as a variable, with variation from point to point being considered error.
    error = np.sqrt(((point_x - ball_x)**2 + (point_y - ball_y)**2 + (point_z - ball_z)**2)) - radius

    return (error)

vars = np.array([0,0,0,0,0,0,0,0])

vars[0] = 0.0 #ball_runout
vars[1] = 0.0 #ball_runout_angle
vars[2] = 0.0 #radius
vars[3] = 0.0 #spindle_centerline_offset
vars[4] = 0.0 #x_offset
vars[5] = 0.0 #y_home_offset
vars[6] = 0.0 #z_home_offset
vars[7] = 0.0 #c_home_offset

p, success, infodict,mesg,ier  = scipy.optimize.leastsq(arbor_leastsq, vars, args=(x, y, z, a, c), full_output=1)

print(mesg)
print("\nsolver converged in ", infodict['nfev'], " iterations\n")

p_three_decimals = ["%.3f" % v for v in p]

print('Results:')
print('ball_runout \t\t\t=\t', p_three_decimals[0], "mm")
print('ball_runout_angle \t\t=\t', p_three_decimals[1], "deg")
print('combined_radius \t\t=\t', p_three_decimals[2], "mm")
print('spindle_centerline_offset \t=\t', p_three_decimals[3], "mm")
print('x_offset \t\t\t=\t', p_three_decimals[4], "mm")
print('y_home_offset \t\t\t=\t', p_three_decimals[5], "mm")
print('z_home_offset \t\t\t=\t', p_three_decimals[6], "mm")
print('c_home_offset \t\t\t=\t', p_three_decimals[7], "deg")

## Results

Here we do some analysis on the result by plotting a histogram. 

This histogram and standard deviaton give an idea of how well the solution conforms to the data.

In [ ]:
%matplotlib notebook

# best fit of data
(mu, sigma) = norm.fit(infodict['fvec'])

# the histogram of the data
n, bins, patches = plt.hist(infodict['fvec'], 15, density=True)

# add a 'best fit' line
y = scipy.stats.norm.pdf( bins, mu, sigma)
l = plt.plot(bins, y, 'r--', linewidth=2)

plt.xlabel('Residual error (mm)')
plt.ylabel('Number of points (normalised)')
plt.title(r'$\mathrm{Histogram\ of\ Residual\ Errors:}\ \mu=%.3f,\ \sigma=%.3f$' %(mu, sigma))

plt.show()

We can also try and visualise how the error is distributed with a 3D plot.

In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D

Axes3D = Axes3D  # pycharm auto import
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')

# Normalised [0,1]
fvec_normalised = (infodict['fvec'] - np.min(infodict['fvec']))/np.ptp(infodict['fvec'])

ax.scatter3D(data[:,0], data[:,1], data[:,2], s = 20000*np.abs(infodict['fvec']), c = fvec_normalised, cmap = 'cool')

ax.set_xlabel('X (mm)')
ax.set_ylabel('Y (mm)')
ax.set_zlabel('Z (mm)')

Here the size of the point relates to the magnatude of the residual error, and the colour indicates positive or negative deviation of the radius measured at each point.

## Save New Offsets to the Machine

Now we can save the results to the machine.

First copy the result to machine variables.

In [ ]:
from amcnc import variable
ipf100 = variable.CncFloat('IPF100')
ipf101 = variable.CncFloat('IPF101')
ipf102 = variable.CncFloat('IPF102')
ipf103 = variable.CncFloat('IPF103')

ipf100.value = p[3] #spindle_centerline_offset
ipf101.value = p[5] #y_home_offset
ipf102.value = p[6] #z_home_offset
ipf103.value = p[7] #c_home_offset


Now run a part program to review changes and save the new offsets to the database

In [ ]:
from amcnc import program

status = program.run(os.path.abspath(r'write_results.pp'))

In [ ]:
def arbor_and_spindle_face(vars, data, num_arbor_pts):
    """This function uses data from the arbor as well as data from the spindle front face 
    to solve for all the machine offsets"""
    
    ball_runout = vars[0]
    ball_runout_angle = vars[1]
    ball_radius = vars[2]
    arbor_radius = vars[3]
    spindle_centerline_offset = vars[4]
    x_offset = vars[5]
    y_home_offset = vars[6]
    z_home_offset = vars[7]
    c_home_offset = vars[8]
    spindle_ref_position= vars[9]
    
    x = data[:, 0]
    y = data[:, 1]
    z = data[:, 2]
    a = data[:, 3]
    c = data[:, 4]

    ball_runout_angle_radians = (ball_runout_angle/360.0)*2*pi
    c_home_offset_radians = (c_home_offset/360.0)*2*pi
    
    for i in range(0, data.shape[0]):

        a_angle_radians = (a[i]/360.0)*2*pi
        
        ball_x = x[i] + x_offset
        ball_y = y[i] + y_home_offset + ball_runout*sin(a_angle_radians + ball_runout_angle_radians)
        ball_z = z[i] + z_home_offset + ball_runout*cos(a_angle_radians + ball_runout_angle_radians)

        c_angle_radians = (c[i]/360.0)*2*pi

        #find equation of the c-axis vector
        ui = -sin(c_angle_radians + c_home_offset_radians)
        uj = cos(c_angle_radians + c_home_offset_radians)
        uk = 0

        #coords of the spindle axis origin
        x0 = cos(c_angle_radians + c_home_offset_radians)*(-spindle_centerline_offset)
        y0 = sin(c_angle_radians + c_home_offset_radians)*(-spindle_centerline_offset)
        z0 = 0
        
        if i < num_arbor_pts:
            #find the closest point on the line
            point_x = x0 + ((ball_x-x0)*ui+(ball_y-y0)*uj+(ball_z-z0)*uk)*ui;
            point_y = y0 + ((ball_x-x0)*ui+(ball_y-y0)*uj+(ball_z-z0)*uk)*uj;
            point_z = z0 + ((ball_x-x0)*ui+(ball_y-y0)*uj+(ball_z-z0)*uk)*uk;

            #the error for each point is the distance from the closest point on the c-axis vector, minus the current value for radius
            error[i] = np.sqrt(((point_x - ball_x)**2 + (point_y - ball_y)**2 + (point_z - ball_z)**2)) - (ball_radius + arbor_radius)

        else:
            #ball center should be ball_radius + spindle_ref_position from spindle face
            error[i] = np.sqrt(((x0 - ball_x)**2 + (y0 - ball_y)**2 + (z0 - ball_z)**2)) - (ball_radius + spindle_ref_position)

        
    return (error)